In [2]:
!pip3 install highway_env


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


In [3]:
import gymnasium
import highway_env
import numpy as np
import pprint

import torch
import torch.nn as nn
import torch.nn.functional as F
import copy



In [ ]:


env = gymnasium.make("highway-v0", render_mode='rgb_array')
# pprint.pprint(env.unwrapped.config)
env.unwrapped.config["lanes_count"] = 3
env.unwrapped.config["duration"] = 10
env.unwrapped.config["vehicles_density"] = 2 # 3
env.unwrapped.config["vehicles_count"] = 10

env.unwrapped.config["action"]["type"] = "DiscreteAction"
ACTION_SIZE = 9
# ACTION_SIZE = 5
observation = {
        "type": "Kinematics",
        "vehicles_count": 5,
        "features": ["x", "y", "vx", "vy", "cos_h"],
        "features_range": {
            "x": [-100, 100],
            "y": [-100, 100],
            "vx": [-20, 20],
            "vy": [-20, 20]
        },
        "absolute": True,
        # "absolute": True,
        "order": "sorted"
    }
env.unwrapped.config["observation"] = observation



In [29]:

class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=50):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.layer_1 = nn.Linear(input_size, hidden_size)
        self.norm_1 = nn.BatchNorm1d(hidden_size)
        self.layer_2 = nn.Linear(hidden_size, hidden_size)
        self.norm_2 = nn.BatchNorm1d(hidden_size)
        self.layer_3 = nn.Linear(hidden_size, output_size)
        self.norm_3 = nn.BatchNorm1d(output_size)

    def forward(self, obs, batch_size=1):
        batch_norm_on = batch_size != 1
        if obs is None:
            retval = torch.tensor([[1/self.output_size] * self.output_size] * batch_size)
            return retval
        
       
        x = torch.tensor(obs)
        x = x.view(-1, self.input_size)


        x = self.layer_1(x)
        if batch_norm_on:
            x = self.norm_1(x)
        x = F.sigmoid(x)
        x = self.layer_2(x)
        if batch_norm_on:
            x = self.norm_2(x)
        x = F.sigmoid(x)
        x = self.layer_3(x)
        if batch_norm_on:
            x = self.norm_3(x)
        x = F.softmax(x, dim=1)
        
        return x
    


In [30]:
import random


class ReplayMemory(object):
    def __init__(self, capacity=1000):
        self.capacity = capacity
        self.memory = []

    def push(self, *args):
        """Save a transition"""
        self.memory.append(args)
        if len(self.memory) > self.capacity:
            self.memory.pop(0)

    def sample(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        states = []
        actions = []
        rewards = []
        for i, memory in enumerate(batch):
            states.append(torch.tensor(memory[0]))
            actions.append(torch.tensor(memory[1]))
            rewards.append(torch.tensor(memory[2]))
        states = torch.stack(states)
        actions = torch.stack(actions)
        rewards = torch.stack(rewards)
            
        return states, actions, rewards

    def __len__(self):
        return len(self.memory)

In [46]:
def select_action(obs, policy_net):
    if obs is None:
        return 0
    policy = policy_net(obs)
    policy = policy.detach().numpy().flatten()
    return np.random.choice(len(policy), p=policy)
    # policy.numpy()
    # return torch.argmax(policy, dim=1).item()

def optimize_model(policy_net, value_net, memory, BATCH_SIZE, GAMMA, optimizer):

    if len(memory) < BATCH_SIZE:
        return
    # print('learn')
    state_batch, action_batch, reward_batch = memory.sample(BATCH_SIZE)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net

    state_action_values = policy_net(state_batch, batch_size=BATCH_SIZE).gather(1, action_batch.unsqueeze(1))
    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.

    next_state_values = torch.max(value_net(state_batch, batch_size=BATCH_SIZE), dim=1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    print(f'Loss: {loss:.2f}')
    # [batch_size, 1, 1]

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()
    


In [48]:
import time
OBS_SIZE = env.unwrapped.config["observation"]["vehicles_count"] * len(env.unwrapped.config["observation"]["features"])
env.unwrapped.config["duration"] = 20

EPOCHS = 100
batch_size = 16
GAMMA = 0.99
TAU = 0.005
LR = 1e-2

policy_net = Model(OBS_SIZE, ACTION_SIZE)
value_net = Model(OBS_SIZE, ACTION_SIZE)
optimizer = torch.optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)


all_parameters = list(policy_net.parameters()) + list(value_net.parameters())
optimizer_random = torch.optim.Adam(all_parameters, lr=0.1)

env.reset()
# while(True):
#     env.render()

memory = ReplayMemory()
for epoch in range(EPOCHS):
    print(f'epoch: {epoch}')
    env.reset()
    obs = None
    done = truncated = False
    
    total_reward = 0
    while not (done or truncated):
        action = select_action(obs, policy_net)
        next_obs, reward, done, truncated, info = env.step(action)

        if info["rewards"]["on_road_reward"] == 0:
            pass# reward -= 10
        # "features": ["x", "y", "vx", "vy", "cos_h"],
        # reward += next_obs[0][1] * 10
        if done or truncated:
            break
        if obs is not None:
            # print(f'sub: {abs(next_obs[0][4] - obs[0][4])}')
            # reward -= abs(next_obs[0][4] - obs[0][4]) * 0.2
            memory.push(obs, action, reward)
        obs = next_obs
        
        total_reward += reward
        # print(total_reward)
        
        if epoch < 5: # 5
            target = torch.Tensor([[1 / ACTION_SIZE] * ACTION_SIZE])
            policy = policy_net(obs)
            values = value_net(obs)
            criterion = torch.nn.CrossEntropyLoss()
            loss_policy = criterion(policy, target)
            
            loss_values = criterion(policy, values)
            loss = loss_policy + loss_values
            optimizer_random.zero_grad()
            loss.backward()
            optimizer.step()

            

        else:
            print(f'policy: {policy_net(obs)}')
            print(f'value: {value_net(obs)}')
            optimize_model(policy_net, value_net, memory, batch_size, GAMMA, optimizer)

            value_net_state_dict = value_net.state_dict()
            policy_net_state_dict = policy_net.state_dict()
            for key in policy_net_state_dict:
                value_net_state_dict[key] = value_net_state_dict[key]*TAU + value_net_state_dict[key]*(1-TAU)
            value_net.load_state_dict(value_net_state_dict)
            env.render()




epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
policy: tensor([[0.1246, 0.1493, 0.4039, 0.2568, 0.0653]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1594, 0.1861, 0.2963, 0.2120, 0.1463]], grad_fn=<SoftmaxBackward0>)
policy: tensor([[0.1245, 0.1493, 0.4040, 0.2568, 0.0653]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1594, 0.1860, 0.2963, 0.2120, 0.1463]], grad_fn=<SoftmaxBackward0>)
epoch: 6
policy: tensor([[0.1247, 0.1493, 0.4038, 0.2569, 0.0653]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1594, 0.1861, 0.2963, 0.2118, 0.1463]], grad_fn=<SoftmaxBackward0>)
policy: tensor([[0.1246, 0.1492, 0.4041, 0.2568, 0.0653]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1594, 0.1860, 0.2964, 0.2119, 0.1463]], grad_fn=<SoftmaxBackward0>)
epoch: 7
policy: tensor([[0.1247, 0.1493, 0.4038, 0.2568, 0.0653]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1594, 0.1862, 0.2961, 0.2120, 0.1463]], grad_fn=<SoftmaxBackward0>)
policy: tensor([[0.1246, 0.1493, 0.4041, 0.2567, 0.0653]], grad

/var/folders/23/10qz4m5j15qdd0tp45tczpw40000gn/T/ipykernel_98788/1276593815.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(obs)
/var/folders/23/10qz4m5j15qdd0tp45tczpw40000gn/T/ipykernel_98788/1276593815.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(obs)


policy: tensor([[0.1181, 0.1459, 0.3884, 0.2802, 0.0674]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1594, 0.1860, 0.2964, 0.2119, 0.1463]], grad_fn=<SoftmaxBackward0>)
Loss: 0.39
epoch: 8
policy: tensor([[0.1133, 0.1429, 0.3921, 0.2849, 0.0667]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1595, 0.1862, 0.2960, 0.2120, 0.1463]], grad_fn=<SoftmaxBackward0>)
Loss: 0.35


/var/folders/23/10qz4m5j15qdd0tp45tczpw40000gn/T/ipykernel_98788/1276593815.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(obs)
/var/folders/23/10qz4m5j15qdd0tp45tczpw40000gn/T/ipykernel_98788/1276593815.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(obs)


policy: tensor([[0.1122, 0.1406, 0.3965, 0.2844, 0.0663]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1593, 0.1861, 0.2963, 0.2119, 0.1463]], grad_fn=<SoftmaxBackward0>)
Loss: 0.30
policy: tensor([[0.1123, 0.1389, 0.4005, 0.2833, 0.0650]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1593, 0.1858, 0.2967, 0.2120, 0.1462]], grad_fn=<SoftmaxBackward0>)
Loss: 0.30


...

/var/folders/23/10qz4m5j15qdd0tp45tczpw40000gn/T/ipykernel_98788/1276593815.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(obs)
/var/folders/23/10qz4m5j15qdd0tp45tczpw40000gn/T/ipykernel_98788/1276593815.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(obs)


policy: tensor([[0.0746, 0.0885, 0.5259, 0.2811, 0.0298]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1594, 0.1861, 0.2963, 0.2119, 0.1463]], grad_fn=<SoftmaxBackward0>)
Loss: 0.37
policy: tensor([[0.0715, 0.0861, 0.5499, 0.2640, 0.0285]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1595, 0.1862, 0.2960, 0.2120, 0.1464]], grad_fn=<SoftmaxBackward0>)
Loss: 0.49


/var/folders/23/10qz4m5j15qdd0tp45tczpw40000gn/T/ipykernel_98788/1276593815.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(obs)
/var/folders/23/10qz4m5j15qdd0tp45tczpw40000gn/T/ipykernel_98788/1276593815.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(obs)


epoch: 98
policy: tensor([[0.0721, 0.0857, 0.5550, 0.2586, 0.0287]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1593, 0.1861, 0.2964, 0.2119, 0.1463]], grad_fn=<SoftmaxBackward0>)
Loss: 0.21
epoch: 99
policy: tensor([[0.0704, 0.0875, 0.5537, 0.2598, 0.0286]], grad_fn=<SoftmaxBackward0>)
value: tensor([[0.1594, 0.1861, 0.2962, 0.2120, 0.1463]], grad_fn=<SoftmaxBackward0>)
Loss: 0.39


/var/folders/23/10qz4m5j15qdd0tp45tczpw40000gn/T/ipykernel_98788/1276593815.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(obs)
/var/folders/23/10qz4m5j15qdd0tp45tczpw40000gn/T/ipykernel_98788/1276593815.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(obs)


In [50]:

GENERATIONS = 1000
SEEDS_PER_GEN = 2
MODELS = 10
OBS_SIZE = env.unwrapped.config["observation"]["vehicles_count"] * len(env.unwrapped.config["observation"]["features"])

models = [Model(OBS_SIZE, ACTION_SIZE) for i in range(MODELS)]

for generation in range(GENERATIONS):
    env.unwrapped.config["duration"] = 8 + 0.5 * generation
    print(f'Generation: {generation}')
    # create models

    scores = [0 for i in range(MODELS)]

    for i in range(SEEDS_PER_GEN):
        seed = np.random.randint(1_000_000)
        for model_num in range(MODELS):
            print(f'{model_num} ', end='')
            render = False
            if model_num % 1 == 0:
                render = True
            model = models[model_num]

            env.reset(seed=seed)
            done = truncated = False
            obs = None
            score = 0
            while not (done or truncated):
                probabilities = model(obs).view(-1).detach().numpy()
                action = np.argmax(probabilities)
                # action = np.random.choice(ACTION_SIZE, p=probabilities)
                obs, reward, done, truncated, info = env.step(action)
                if info["rewards"]["on_road_reward"] == 0:
                    # reward -= 10
                    done = True
                if render:
                    env.render()
                # score += reward
                score += obs[0][2]
            print(score)
            # print(obs)

            # if done:
            #     score -= 10

            scores[model_num] += score
            # scores[model_num] += score
        print()
    best_score = max(scores)
    best_model_num = scores.index(best_score)
    best_model = models[best_model_num]
    print(f'Best model_num: {best_model_num} | score: {best_score}\n')
    models = [copy.deepcopy(best_model) for i in range(MODELS)]
    change = 0.01 * (0.999 ** generation)
    for model in models:
        for param in model.parameters():
            param.data += change * torch.randn_like(param)
    torch.save(best_model.state_dict(), f'model_gen{generation}')





Generation: 0
0 8.0
1 1.7637636065483093
2 6.509786248207092
3 1.7637636065483093
4 6.509786248207092
5 1.7637636065483093
6 4.863429844379425
7 6.509786248207092
8 8.0
9 1.7637636065483093

0 1.4644529819488525
1 1.4755589962005615
2 1.4750087559223175
3 1.4755589962005615
4 1.4750087559223175
5 1.4755589962005615
6 1.48556450009346
7 1.4750087559223175
8 1.4644529819488525
9 1.4755589962005615

Best model_num: 0 | score: 9.464452981948853

Generation: 1
0 1.6037171483039856
1 1.6037171483039856
2 1.6037171483039856
3 1.6037171483039856
4 1.6037171483039856
5 1.6037171483039856
6 1.6037171483039856
7 1.6037171483039856
8 1.6037171483039856
9 1.6037171483039856

0 4.617260575294495
1 4.617260575294495
2 4.617260575294495
3 4.617260575294495
4 4.617260575294495
5 4.617260575294495
6 4.617260575294495
7 4.617260575294495
8 4.617260575294495
9 4.617260575294495

Best model_num: 0 | score: 6.22097772359848

Generation: 2
0 9.0
1 9.0
2 9.0
3 9.0
4 9.0
5 9.0
6 9.0
7 9.0
8 9.0
9 9.0

0 9.0
1 

AttributeError: 'NoneType' object has no attribute 'get_image'